In [1]:
# 设计一些toy data example去支撑我们的设计
# ①假设一：使用方形网格设计（后续可以改成六边形，其实也不一定有必要）
# ② 假设二：区域设定为6公里 × 6公里，设定一个合理的骑手速度，骑手量与订单量（骑手量和订单量可以作为变量调节）
# ③ 假设三：区域6km × 6km，网格300m × 300m，形成一个20 × 20 = 400的网格区域，纬度为30.70 -- 30.76,经度121.10--121.16，
#一个小区域0.0005 × 0.0005，比如30.70 --30.703,121.10 -- 121.103。即0.06为6km，0.003为300m。
# ④ 假设四：骑手速度：12.11km/h,200m/min。这样跑一个对角需要42min，差不多。

### 丁一学长给的经纬度对应关系

In [2]:
# The following is the lat/lon area I used to cover Shanghai.
# AND latitude > 30.7 AND latitude < 31.5
# AND longitude > 121.1 AND longitude < 121.9

# The following is the mapping between grid ID and lat/lon.
# ((FLOOR(t1.latitude*100)-3070)*80+(FLOOR(t1.longitude*100)-12110)) AS grid_id

### 头文件区

In [3]:
from math import sin, asin, cos, radians, fabs, sqrt
EARTH_RADIUS = 6371  # 地球平均半径，6371km

### 函数区

In [4]:
def hav(theta):
    s = sin(theta / 2)
    return s * s


def get_distance_hav(lat0, lng0, lat1, lng1):
    """
     用haversine公式计算球面两点间的距离
    """
    # 经纬度转换成弧度
    lat0 = radians(lat0)
    lat1 = radians(lat1)
    lng0 = radians(lng0)
    lng1 = radians(lng1)

    dlng = fabs(lng0 - lng1)
    dlat = fabs(lat0 - lat1)
    h = hav(dlat) + cos(lat0) * cos(lat1) * hav(dlng)
    distance = 2 * EARTH_RADIUS * asin(sqrt(h))

    return round(distance * 1000)

In [5]:
get_distance_hav(30.70,121.10,30.703,121.10)

334

 ### 确定有多少家商家，多少个骑手，多少个订单

In [6]:
# 区域: latitude:30.70--30.76    longitude: 121.10 -- 121.16    6000m 300m  200m/min

In [7]:
# 将区域分为四个部分：高商高顾（CBD），高商低顾（商业区），低商高顾（居民区），低商低顾（郊区）

In [8]:
# 派单区域按照网格来算，每五分钟更新骑手所在的位置和所在的网格，如果骑手在更新时还没有收到新订单，他会前往最近的CBD区和商业区。

In [9]:
# 每五分钟派一次单，即每五分钟更新一次骑手的状态，骑手的状态分为三种(配送中，等待中，回程中)，
#配送中会记录处在哪个区域内，还有多久离开这个区域，下一个区域名称。
# 等待中会记录处在哪个区域中，还有多少等待时间。
# 回程中会记录处在哪个区域内，还有多久离开这个区域，下一个区域名称。
# 骑手前往临近区域只有两种，直行/斜行，直行300m花费1.5min，斜行400m花费2min。
# 骑手会记录一个路径，在派单的过程中会持续记录路径

In [24]:
# 30个CBD，44个商业区，85个居民区，241个郊区。
# 400个区域，293个商家，1000个骑手，n个订单（订单平均长度3500m，每个骑手每小时一定能处理3单）

#### 确定每个区域的类型

In [50]:
# 区域的编号为：x + 20*y  (x,y都是0-19)

In [51]:
# 1 1 2 2
# 1 2 1 3
# 2 3 4 2
# 1 2 1 2
CBDList = []
# 1 2 1 2
# 2 3 2 5
# 2 3 8 1
# 2 5 3 2
commerceList = []
# 4 4 3 8
# 13 7 5 3
# 2 3 6 4
# 5 6 5 7
residentList = []

suburbList = []

In [ ]:
CBDNumList = [1,1,2,2,1,2,1,3,2,3,4,2,1,2,1,2]
commerceNumList = [1,2,1,2,2,3,2,5,2,3,8,1,2,5,3,2]
residentNumList = [4,4,3,8,13,7,5,3,2,3,6,4,5,6,5,7]
suburbNumList = [19,18,19,13,9,13,17,14,19,16,7,18,17,12,16,14]

In [ ]:
fourdistricttype = [30,44,85,241]
nodelist = []
for x in range(4):
    for y in range(4):
        xlist = list(range(x*5,x*5+5))
        ylist = list(range(y*5,x*5+5))
        nodedistrictlist = []
        for nodex in xlist:
            for nodey in ylist:
                nodedistrictlist.append(nodex + nodey*20)
        nodedistrictlist = list(sorted(nodedistrictlist))
        nodelist.append(nodedistrictlist)
for numtype in fourdistricttype:
    
    

In [ ]:
nodelist

In [37]:
fourdistricttype = [30,44,85,241]
nodelist = []
for x in range(4):
    for y in range(4):
        xlist = list(range(x*5,x*5+5))
        ylist = list(range(y*5,x*5+5))
        nodedistrictlist = []
        for nodex in xlist:
            for nodey in ylist:
                nodedistrictlist.append(nodex + nodey*20)
        nodedistrictlist = list(sorted(nodedistrictlist))
        nodelist.append(nodedistrictlist)
for numtype in fourdistricttype:
    
    

In [38]:
nodelist

[[0,
  1,
  2,
  3,
  4,
  20,
  21,
  22,
  23,
  24,
  40,
  41,
  42,
  43,
  44,
  60,
  61,
  62,
  63,
  64,
  80,
  81,
  82,
  83,
  84],
 [],
 [],
 [],
 [5,
  6,
  7,
  8,
  9,
  25,
  26,
  27,
  28,
  29,
  45,
  46,
  47,
  48,
  49,
  65,
  66,
  67,
  68,
  69,
  85,
  86,
  87,
  88,
  89,
  105,
  106,
  107,
  108,
  109,
  125,
  126,
  127,
  128,
  129,
  145,
  146,
  147,
  148,
  149,
  165,
  166,
  167,
  168,
  169,
  185,
  186,
  187,
  188,
  189],
 [105,
  106,
  107,
  108,
  109,
  125,
  126,
  127,
  128,
  129,
  145,
  146,
  147,
  148,
  149,
  165,
  166,
  167,
  168,
  169,
  185,
  186,
  187,
  188,
  189],
 [],
 [],
 [10,
  11,
  12,
  13,
  14,
  30,
  31,
  32,
  33,
  34,
  50,
  51,
  52,
  53,
  54,
  70,
  71,
  72,
  73,
  74,
  90,
  91,
  92,
  93,
  94,
  110,
  111,
  112,
  113,
  114,
  130,
  131,
  132,
  133,
  134,
  150,
  151,
  152,
  153,
  154,
  170,
  171,
  172,
  173,
  174,
  190,
  191,
  192,
  193,
  194,
  210,
